# Objetivo

En este trabajo vamos a tratar de implementar el modelo de evaluación de riesgo de la epidemia desarrollado por el equipo de la [Computational Biology and Complex Systems; Universitat Politècnica de Catalunya - BarcelonaTech](https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf) . Para obtener los datos epidemiológicos nos apoyamos en el grupo de trabajo [#escovid19data](https://github.com/montera34/escovid19data).


@author: Manuel H. Arias 

@Twitter: @walyt

@mail: mharias@me.com



## Importación de librerias estándares para DAE

In [19]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import urllib.request

import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

## Descargamos la información

Hacemos en primer lugar una actualización de parámetros y preparación de variables que necesitaremos durante el ejercicio

In [20]:
pd.options.display.max_rows = 999 #Variable de contexto para permitir la presentación de datos por pantalla
pd.set_option('display.max_columns', None)

In [21]:
#url de este notebook en github
github_url = 'https://github.com/mharias/covid/blob/master/graficos_escovid19data.ipynb'

In [22]:
hashtag_fuente='#escovid19data' #hashtag de la fuente para los créditos

In [23]:
#url de la fuente de datos
path_montera34='https://github.com/montera34/escovid19data/blob/master/data/output/covid19-provincias-spain_consolidated.csv?raw=true'

Leemos los datos en un `pandas`

In [24]:
df = pd.read_csv(path_montera34)

veamos una rápida descripción de la información:

In [25]:
df.describe()

,ine_code,new_cases,PCR,TestAc,activos,hospitalized,hospitalized_accumulated,hospitalized_new,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,deceased_hosp_eus,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_hosp_eus,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,daily_deaths_avg7_hosp_eus,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14
count,19113.000000,6314.000000,5167.000000,1349.000000,2278.000000,13281.000000,815.000000,812.000000,13144.000000,14596.000000,9413.000000,10374.000000,9333.000000,585.000000,18928.000000,18928.000000,18928.000000,18928.000000,18928.000000,18928.000000,1.911300e+04,9413.000000,13144.000000,14596.000000,13281.000000,8433.000000,8882.000000,9707.000000,10024.00000,11881.000000,11514.000000,11317.000000,10953.000000,14537.000000,582.000000,13762.0,13714.000000,14214.000000,564.000000,19113.000000,18616.000000,18616.000000,11996.000000
mean,26.503584,94.072537,137.439959,8377.395107,1084.552239,198.390558,15599.975460,969.108374,29.817407,692.850713,11357.344099,14209.240505,3801.262065,391.068376,98.995139,87.341822,0.248098,11.137944,0.158495,0.108781,9.042921e+05,1079.061459,3.311039,67.044841,22.908704,1881.847622,927.462058,1875.389616,931.82103,116.539180,116.783759,128.253624,127.294184,4.078765,1.475945,inf,4.106001,4.026755,1.462766,21.591587,100.181215,88.486673,216.493064
std,15.013877,177.796983,310.215130,22898.617898,1547.602621,698.019237,24577.877953,1427.051716,84.295494,1970.051256,25044.987824,42258.189302,7293.827087,243.192336,287.304103,264.879105,1.449871,57.914229,1.737191,1.298625,1.177452e+06,1319.794934,3.794338,59.302004,37.367801,3704.312541,1865.360714,4906.413233,2462.06911,264.604565,241.609351,373.403230,337.748516,17.992303,1.765594,NaN,18.047313,17.534583,1.358153,106.735881,278.041297,255.779778,242.334018
min,1.000000,-2.000000,-1.000000,0.000000,-1.000000,0.000000,3.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,130.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.477700e+04,0.000000,0.000000,0.000000,0.000000,-54.000000,-82.000000,-62.000000,-99.00000,-106.000000,-12.000000,-84.000000,-12.700000,-22.000000,0.000000,-30.8,-1.300000,-0.600000,0.000000,-4.000000,0.000000,0.000000,-6.900000
25%,13.000000,7.000000,9.000000,2.000000,209.000000,18.000000,288.000000,215.250000,3.000000,116.000000,1195.000000,1769.000000,534.000000,219.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.167980e+05,171.560000,0.600000,16.170000,4.100000,213.000000,97.250000,70.000000,32.00000,7.000000,10.100000,4.000000,4.700000,0.000000,0.000000,0.0,0.000000,0.100000,0.300000,0.000000,1.600000,1.400000,21.500000
50%,26.000000,36.000000,53.000000,11.000000,582.500000,67.000000,1212.000000,300.000000,10.000000,254.000000,3237.000000,3998.500000,1491.000000,285.000000,18.000000,16.000000,0.000000,0.000000,0.000000,0.000000,5.810780e+05,519.600000,2.110000,47.715000,13.870000,737.000000,355.000000,536.000000,261.50000,40.000000,45.100000,32.000000,37.000000,1.000000,1.000000,0.2,1.000000,1.000000,1.000000,3.000000,19.900000,18.100000,147.200000
75%,40.000000,95.000000,142.000000,297.000000,1219.750000,171.000000,41231.500000,1159.250000,29.000000,470.000000,10918.000000,9221.000000,3257.000000,594.000000,90.000000,80.000000,0.000000,1.000000,0.000000,0.000000,1.022800e+06,1474.430000,4.720000,108.590000,29.070000,1919.000000,930.000000,1572.500000,781.00000,115.000000,119.850000,111.000000,110.800000,3.000000,2.000000,1.0,3.000000,3.100000,2.300000,16.000000,92.300000,83.325000,321.025000
max,52.000000,2332.000000,4422.000000,84626.000000,14617.000000,15227.000000,77607.000000,65

y un muestreo de valores y de algunas columnas de interés:

In [26]:
df.head()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,hospitalized_accumulated,hospitalized_new,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,deceased_hosp_eus,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_hosp_eus,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,daily_deaths_avg7_hosp_eus,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments
0,2020-01-01,Albacete,2,Castilla - La Mancha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,388167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01,Alicante/Alacant,3,Comunitat Valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1858683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,Araba/Álava,1,País Vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,331549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,Asturias,33,"Asturias, Principado de",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1022800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.tail()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,hospitalized_accumulated,hospitalized_new,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,deceased_hosp_eus,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_hosp_eus,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,daily_deaths_avg7_hosp_eus,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments
19108,2021-01-03,Segovia,40,Castilla y León,51.0,NaN,NaN,NaN,17.0,NaN,NaN,10.0,262.0,NaN,8774.0,1211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153129,NaN,6.53,171.10,11.10,NaN,NaN,542.0,398.0,51.0,56.9,51.0,56.9,0.0,NaN,0.0,0.0,0.0,NaN,0,NaN,NaN,353.9,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
19109,2021-01-03,Soria,42,Castilla y León,12.0,NaN,NaN,NaN,7.0,NaN,NaN,5.0,181.0,NaN,5925.0,760.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88636,NaN,5.64,204.21,7.90,NaN,NaN,125.0,90.0,12.0,12.9,12.0,12.9,0.0,NaN,0.0,0.0,0.3,NaN,2,NaN,NaN,141.0,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
19110,2021-01-03,Valencia/València,46,Comunitat Valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2565124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,NaN,NaN,NaN,NaN,NaN,No hay datos
19111,2021-01-03,Valladolid,47,Castilla y León,96.0,NaN,NaN,NaN,73.0,NaN,NaN,42.0,820.0,NaN,31825.0,4750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,519546,NaN,8.08,157.83,14.05,NaN,NaN,892.0,591.0,96.0,84.3,96.0,84.4,1.0,NaN,0.1,0.7,1.1,NaN,8,NaN,NaN,171.7,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
19112,2021-01-03,Zamora,49,Castilla y León,8.0,NaN,NaN,NaN,38.0,NaN,NaN,8.0,337.0,NaN,8855.0,1408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172539,NaN,4.64,195.32,22.02,NaN,NaN,170.0,107.0,8.0,15.3,8.0,15.3,0.0,NaN,0.0,0.7,1.0,NaN,7,NaN,NaN,98.5,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN


In [28]:
df['province'].unique()

array(['Albacete', 'Alicante/Alacant', 'Almería', 'Araba/Álava',
       'Asturias', 'Ávila', 'Badajoz', 'Balears, Illes', 'Barcelona',
       'Bizkaia', 'Burgos', 'Cáceres', 'Cádiz', 'Cantabria',
       'Castellón/Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Gipuzkoa', 'Girona', 'Granada',
       'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Lugo', 'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Navarra',
       'Ourense', 'Palencia', 'Palmas, Las', 'Pontevedra', 'Rioja, La',
       'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla',
       'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia/València',
       'Valladolid', 'Zamora', 'Zaragoza'], dtype=object)

Vamos a formatear correctamente la columna `Date`, para posteriormente ordenar el $pandas$ por fecha creciente:

In [29]:
df['date']=pd.to_datetime(df['date'])

In [30]:
df.columns

Index(['date', 'province', 'ine_code', 'ccaa', 'new_cases', 'PCR', 'TestAc',
       'activos', 'hospitalized', 'hospitalized_accumulated',
       'hospitalized_new', 'intensive_care', 'deceased', 'cases_accumulated',
       'cases_accumulated_PCR', 'recovered', 'deceased_hosp_eus', 'num_casos',
       'num_casos_prueba_pcr', 'num_casos_prueba_test_ac',
       'num_casos_prueba_ag', 'num_casos_prueba_elisa',
       'num_casos_prueba_desconocida', 'poblacion', 'cases_per_cienmil',
       'intensive_care_per_1000000', 'deceassed_per_100000',
       'hospitalized_per_100000', 'cases_14days', 'cases_7days',
       'cases_PCR_14days', 'cases_PCR_7days', 'daily_cases',
       'daily_cases_avg7', 'daily_cases_PCR', 'daily_cases_PCR_avg7',
       'daily_deaths', 'daily_deaths_hosp_eus', 'daily_deaths_inc',
       'daily_deaths_avg3', 'daily_deaths_avg7', 'daily_deaths_avg7_hosp_eus',
       'deaths_last_week', 'num_casos_avg7', 'num_casos_prueba_pcr_avg7',
       'ia14', 'source_name', 'source'

# Gráfico de BiocomSC

## preparamos el Pandas

In [31]:
# Gráfico de BiocomSC


provincias=['Madrid']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=3,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='incidencia_acumulada_14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'daily_cases_PCR'
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
for provincia in provincias:
    
    datos.loc[datos['province']==provincia,AI]=\
    datos.loc[datos['province']==provincia,casos_PCR_14]\
    /datos.loc[datos['province']==provincia,'poblacion']*100000

    datos.loc[datos['province']==provincia,clave_ratio] = \
    datos.loc[datos['province']==provincia][AI].pct_change(periods=7).add(1)    
    
    datos.loc[datos['province']==provincia,clave_casos_popu]=\
    datos.loc[datos['province']==provincia,casos_nuevos]\
    /datos.loc[datos['province']==provincia,'poblacion']*1000000

#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=3,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),'daily_cases_PCR'].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),'daily_cases_PCR'].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

In [32]:
datos

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,hospitalized_accumulated,hospitalized_new,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,deceased_hosp_eus,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_hosp_eus,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,daily_deaths_avg7_hosp_eus,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments,incidencia_acumulada_14,tasa_ia_7_dias,casos_per_population,p7,p7_avg7
0,2020-03-01,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,62.0,0.0,0.0,332.0,331.0,0.0,1.0,0.0,0.0,6663394,0.80,0.00,0.00,0.00,0.0,52.0,0.0,61.0,26.0,7.4,26.0,8.7,0.0,0.0,0.0,0.0,0.0,0.0,0,126.7,126.3,0.0,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,0.000000,NaN,3.901915,0.000000,NaN
1,2020-03-02,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,109.0,0.0,0.0,388.0,387.0,0.0,1.0,0.0,0.0,6663394,1.44,0.00,0.00,0.00,0.0,95.0,0.0,108.0,43.0,13.6,47.0,15.4,0.0,0.0,0.0,0.0,0.0,0.0,0,174.0,173.4,0.0,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,0.000000,NaN,7.053463,0.000000,NaN
2,2020-03-03,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142.0,158.0,0.0,0.0,408.0,408.0,0.0,0.0,0.0,0.0,6663394,2.13,0.00,0.00,0.00,0.0,140.0,0.0,156.0,46.0,20.0,49.0,22.3,0.0,0.0,0.0,0.0,0.0,0.0,0,224.7,224.1,0.0,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,0.000000,NaN,7.353610,0.000000,NaN
3,2020-03-04,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,199.0,226.0,0.0,0.0,473.0,473.0,0.0,0.0,0.0,0.0,6663394,2.99,0.00,0.02,0.00,0.0,193.0,0.0,219.0,57.0,27.6,68.0,31.3,1.0,0.0,inf,0.0,0.0,0.0,1,281.9,281.4,0.0,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,0.000000,NaN,10.205010,0.000000,NaN
4,2020-03-05,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,311.0,352.0,0.0,0.0,668.0,668.0,0.0,0.0,0.0,0.0,6663394,4.67,0.00,0.03,0.00,310.0,301.0,351.0,340.0,112.0,43.0,126.0,48.6,1.0,0.0,100.0,0.0,0.0,0.0,2,363.3,362.9,5.3,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,5.267586,NaN,18.909283,0.000000,NaN
5,2020-03-06,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,515.0,564.0,0.0,0.0,808.0,808.0,0.0,0.0,0.0,0.0,6663394,7.73,0.00,0.06,0.00,514.0,495.0,563.0,539.0,204.0,70.7,212.0,77.0,2.0,0.0,100.0,1.3,0.0,0.0,4,455.4,455.0,8.4,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,8.449148,NaN,31.815618,0.000000,NaN
6,2020-03-07,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,738.0,810.0,0.0,0.0,867.0,865.0,1.0,1.0,0.0,0.0,6663394,11.08,0.00,0.12,0.00,737.0,711.0,809.0,774.0,223.0,101.6,246.0,110.6,4.0,0.0,100.0,2.3,0.0,0.0,8,563.4,562.9,12.1,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,12.140960,NaN,36.918123,0.000000,0.000000
7,2020-03-08,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,1003.0,1098.0,0.0,0.0,1088.0,1088.0,0.0,0.0,0.0,0.0,6663394,15.05,0.00,0.24,0.00,1002.0,950.0,1097.0,1036.0,265.0,135.7,288.0,148.0,8.0,0.0,100.0,4.7,0.0,0.0,16,671.4,671.0,16.5,Instituto de Salud Carlos III;Consejería de Sa...,https://cnecovid.isciii.es/covid19/resources/a...,0,16.463082,inf,43.221217,0.000000,0.000000
8,2020-03-09,Madrid,28,"Madrid, Comunidad de",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,1508.0,16

In [33]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
#y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
#x0, x1 = 0,300
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    #print (i)
    #ax.set_xlim(0,x1)
    #ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Madrid\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)


lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [34]:
len(datos.index)

308

In [35]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

In [17]:
anim.save('modelo_biocomsc_madrid.mp4',dpi=288)

# Gráfico de BiocomSC Badajoz

In [83]:
provincias=['Barcelona']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=7,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='ia14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'PCR' #Navarra
casos_nuevos = 'new_cases' #badajoz #ojo con elegir el campo correcto...
casos_nuevos = 'daily_cases' #Barcelona
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=7,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),casos_nuevos].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),casos_nuevos].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos[(datos['date']>=datetime(year=2020,month=8,day=1))]
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

In [84]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
x0, x1 = 0,1200
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    
    #print (i)
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Badajoz\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)
    

    
lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [85]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

<ipython-input-84-58ad3c867318>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-84-58ad3c867318>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-84-58ad3c867318>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,


In [82]:
anim.save('videos/modelo_biocomsc_barcelona.mp4',dpi=288)

<ipython-input-79-58ad3c867318>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-79-58ad3c867318>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-79-58ad3c867318>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,
